In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [6]:

# 🔹 1️⃣ Chargement des données déjà transformées (avec ACP appliqué)
train = pd.read_csv("./final_data.csv")
test = pd.read_csv("./final_test_data.csv")
sample_submission = pd.read_csv("../Housing-prices-competition/home-data-for-ml-course/sample_submission.csv")

# 🔹 2️⃣ Définition des features et de la target
y_train = train["SalePrice"]  # Variable cible
X_train = train.drop(columns=["SalePrice"])  # Features après ACP

X_test = test.copy()  # Test après ACP

# 🔹 3️⃣ Définition de la grille de recherche pour GridSearchCV
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, 15],
    "min_samples_split": [2, 5, 10]
}

# 🔹 4️⃣ Recherche des meilleurs hyperparamètres
grid_search = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid,
    cv=3,
    scoring="r2",
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

# 🔹 5️⃣ Récupération des meilleurs paramètres
best_params = grid_search.best_params_
print("✅ Meilleurs paramètres trouvés:", best_params)

# 🔹 6️⃣ Entraînement du modèle avec les meilleurs hyperparamètres
best_rf_model = RandomForestRegressor(
    n_estimators=best_params["n_estimators"],
    max_depth=best_params["max_depth"],
    min_samples_split=best_params["min_samples_split"],
    random_state=42
)
best_rf_model.fit(X_train, y_train)

# 🔹 7️⃣ Évaluation du modèle sur les données d'entraînement
y_train_pred = best_rf_model.predict(X_train)

# Calcul des métriques
train_r2 = r2_score(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

# 🔹 8️⃣ Affichage des performances du modèle
print("\n📊 Performance du modèle sur le training set:")
print(f"📈 Training R² Score: {train_r2:.4f}")
print(f"📉 Training MAE: {train_mae:.2f}")
print(f"📉 Training RMSE: {train_rmse:.2f}")

# 🔹 9️⃣ Prédictions sur les données test
y_test_pred = best_rf_model.predict(X_test)

# 🔹 🔟 Création du fichier de soumission
submission = pd.DataFrame({"Id": sample_submission["Id"], "SalePrice": y_test_pred})
submission.to_csv("submission_final_data.csv", index=False)
print("\n📁 Prédictions enregistrées dans submission_final_data.csv ✅")


✅ Meilleurs paramètres trouvés: {'max_depth': 15, 'min_samples_split': 2, 'n_estimators': 100}

📊 Performance du modèle sur le training set:
📈 Training R² Score: 0.9779
📉 Training MAE: 6659.84
📉 Training RMSE: 11816.23

📁 Prédictions enregistrées dans submission_final_data.csv ✅
